<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-26 12:41:46
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.36 C
-------------------
Today PnL: -86.08 K (-0.63%)
Current PnL: -19.07 L (-12.97%)
CY Booked + Current PnL: -10.34 L (-7.03%)
-------------------
Total profit:  3.40 L
Total loss:  -22.47 L
-------------------
Total Booked + Current PnL: 16.64 L (13.86%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.41%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.18 L (57.38%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HINDUNILVR,2413.81,2723.00,-3.96,H-LC,8.23,301795.0,31448.0,3169.0,2.39,11.63,1.05,12.81,24.0,9.92,2.27,26.22,XY25,NTT,FMCG
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.35,112680.0,8368.0,8282.0,-1.62,8.02,7.35,15.96,9.0,1.01,0.85,26.28,X5K,ATH,METALS
55,MEDANTA,1087.93,1486.00,13.12,H-MC,14.66,161588.0,35388.0,10794.0,-0.24,28.04,6.68,36.59,108.0,3.28,1.22,44.12,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-14.42,H-LC,8.10,170929.0,14752.0,11333.0,-1.04,9.45,6.63,16.71,22.0,1.30,1.29,28.53,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-13.25,M-LC,4.05,215458.0,13978.0,13229.0,-0.42,6.94,6.14,13.51,60.0,1.06,1.62,20.41,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,5.95,H-SC,11.10,126252.0,955.0,58051.0,-1.76,0.76,45.98,47.09,141.0,0.02,0.95,58.01,MH,ATH,POWER
61,RAJOOENG,102.50,143.33,-31.84,H-SC,10.02,103250.0,750.0,40082.0,-0.67,0.73,38.82,39.83,114.0,0.02,0.78,2.23,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-50.29,H-MC,1.96,99741.0,-763.0,31398.0,-1.17,-0.76,31.48,30.49,79.0,-0.02,0.75,31.40,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,3.09,145418.0,-4514.0,19108.0,0.04,-3.01,13.14,9.74,4.0,-0.24,1.09,3.04,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-4.62,H-LC,3.16,189486.0,-10824.0,70792.0,-0.56,-5.40,37.36,29.94,16.0,-0.15,1.43,30.91,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.90,H-LC,3.39,274365.0,8939.0,47849.0,1.69,3.37,17.44,21.40,11.0,0.19,2.07,10.94,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-18.75,H-LC,4.23,217008.0,12508.0,28905.0,-1.07,6.12,13.32,20.25,10.0,0.43,1.63,15.38,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.41,H-LC,6.49,170891.0,-30495.0,118103.0,0.81,-15.14,69.11,43.51,27.0,-0.26,1.29,18.96,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,14.35,H-LC,8.69,206802.0,8706.0,22252.0,3.50,4.39,10.76,15.63,26.0,0.39,1.56,27.20,XY25,ATH,FMCG
69,SFL,1016.35,1287.00,43.03,M-SC,10.29,188792.0,-73426.0,143255.0,3.01,-28.00,75.88,26.63,229.0,-0.51,1.42,17.38,XY24,NTT,MISC
45,INFY,1461.46,2275.00,-16.37,H-LC,11.99,274246.0,12645.0,132982.0,3.00,4.83,48.49,55.67,3.0,0.10,2.06,11.23,X40,BTT,IT
37,HINDUNILVR,2413.81,2723.00,-3.96,H-LC,8.23,301795.0,31448.0,3169.0,2.39,11.63,1.05,12.81,24.0,9.92,2.27,26.22,XY25,NTT,FMCG
73,SONACOMS,578.05,1006.00,-35.24,M-SC,14.05,80622.0,-20537.0,95424.0,2.12,-20.30,118.36,74.03,202.0,-0.22,0.61,13.69,AR,BTT,AUTO


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,214.25,400.00,-11.26,H-SC,8.49,192711.0,-55819.0,271299.0,-3.10,-22.46,140.78,86.70,151.0,-0.21,1.45,28.65,XY24,NTT,BANKS
85,UNIONBANK,123.87,163.00,-19.38,M-LC,11.40,147048.0,6208.0,38277.0,-2.79,4.41,26.03,31.59,66.0,0.16,1.11,32.06,XY24,NTT,BANKS
71,SIEMENS,4406.90,7969.85,-4.32,H-LC,33.18,76938.0,-33234.0,122308.0,-2.75,-30.17,158.97,80.85,15.0,-0.27,0.58,13.32,AR,ATH,ELECTRICAL
60,RAJESHEXPO,517.65,518.00,1631.06,L-SC,8.39,47215.0,-89962.0,90053.0,-2.66,-65.58,190.73,0.07,267.0,-1.00,0.36,17.43,OX40N,NTT,JEWELLERY
8,AWL,325.39,485.00,-66.09,H-MC,6.66,208684.0,-59763.0,191447.0,-2.58,-22.26,91.74,49.05,116.0,-0.31,1.57,7.30,XY24,NTT,FMCG


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,95.73,M-SC,8.62,84331.0,-16446.0,16537.0,-1.89,-16.32,19.61,0.09,245.0,-0.99,0.63,10.79,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,15.47,116937.0,-25542.0,45068.0,-1.01,-17.93,38.54,13.70,163.0,-0.57,0.88,38.02,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.04,114336.0,-29467.0,56219.0,-1.78,-20.49,49.17,18.60,149.0,-0.52,0.86,26.17,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,109.82,M-SC,1.69,137640.0,-36919.0,36956.0,-1.46,-21.15,26.85,0.02,221.0,-1.00,1.04,19.72,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.25,H-SC,3.64,211437.0,-58230.0,94555.0,0.17,-21.59,44.72,13.47,138.0,-0.62,1.59,6.94,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,15.92,245279.0,-50655.0,144567.0,0.13,-17.12,58.94,31.73,1.0,-0.35,1.85,4.72,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.37,H-LC,11.99,274246.0,12645.0,132982.0,3.00,4.83,48.49,55.67,3.0,0.10,2.06,11.23,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,3.09,145418.0,-4514.0,19108.0,0.04,-3.01,13.14,9.74,4.0,-0.24,1.09,3.04,X40,NTT,FMCG
89,VBL,492.64,671.64,-9.65,H-LC,8.64,274488.0,9448.0,86848.0,-1.12,3.56,31.64,36.33,5.0,0.11,2.07,17.34,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.01,H-LC,12.83,196872.0,-19647.0,112552.0,-1.03,-9.07,57.17,42.91,7.0,-0.17,1.48,3.10,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,13.12,H-MC,14.66,161588.0,35388.0,10794.0,-0.24,28.04,6.68,36.59,108.0,3.28,1.22,44.12,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.76,H-LC,11.13,230403.0,45922.0,24446.0,-0.49,24.89,10.61,38.14,38.0,1.88,1.73,38.14,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,13.12,H-MC,14.66,161588.0,35388.0,10794.0,-0.24,28.04,6.68,36.59,108.0,3.28,1.22,44.12,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,9.17,H-MC,4.80,188490.0,23820.0,44144.0,-1.02,14.47,23.42,41.28,89.0,0.54,1.42,23.60,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,5.95,H-SC,11.10,126252.0,955.0,58051.0,-1.76,0.76,45.98,47.09,141.0,0.02,0.95,58.01,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-3.04,H-MC,6.73,203310.0,11568.0,84455.0,-0.15,6.03,41.54,50.08,99.0,0.14,1.53,13.24,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.05,178056.0,27143.0,30163.0,0.46,17.99,16.94,37.97,93.0,0.90,1.34,47.84,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.05,178056.0,27143.0,30163.0,0.46,17.99,16.94,37.97,93.0,0.90,1.34,47.84,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.97,H-SC,12.37,139814.0,16478.0,117290.0,0.02,13.36,83.89,108.45,119.0,0.14,1.05,38.70,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-43.32,M-SC,5.59,95858.0,4360.0,74395.0,-2.36,4.76,77.61,86.07,223.0,0.06,0.72,38.67,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-11.07,M-LC,7.76,156345.0,5400.0,104048.0,-0.66,3.58,66.55,72.51,53.0,0.05,1.18,9.82,XR,NTT,IT
51,KPIGREEN,497.21,731.35,5.95,H-SC,11.10,126252.0,955.0,58051.0,-1.76,0.76,45.98,47.09,141.0,0.02,0.95,58.01,MH,ATH,POWER


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.09,82253.0,-31296.0,71346.0,-0.35,-27.56,86.74,35.27,268.0,-0.44,0.62,101.32,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.33,H-SC,16.16,89913.0,-11050.0,105585.0,-1.48,-10.94,117.43,93.63,148.0,-0.10,0.68,33.92,SR,ATH,CHEMICALS
54,MASFIN,326.60,398.61,-18.70,H-SC,12.55,92565.0,-5415.0,27020.0,-1.28,-5.53,29.19,22.05,152.0,-0.20,0.70,34.55,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.35,112680.0,8368.0,8282.0,-1.62,8.02,7.35,15.96,9.0,1.01,0.85,26.28,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,85.53,H-SC,15.47,116937.0,-25542.0,45068.0,-1.01,-17.93,38.54,13.70,163.0,-0.57,0.88,38.02,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.09,82253.0,-31296.0,71346.0,-0.35,-27.56,86.74,35.27,268.0,-0.44,0.62,101.32,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-7.76,H-LC,11.13,230403.0,45922.0,24446.0,-0.49,24.89,10.61,38.14,38.0,1.88,1.73,38.14,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.05,178056.0,27143.0,30163.0,0.46,17.99,16.94,37.97,93.0,0.90,1.34,47.84,AR,ATH,AUTO
69,SFL,1016.35,1287.00,43.03,M-SC,10.29,188792.0,-73426.0,143255.0,3.01,-28.00,75.88,26.63,229.0,-0.51,1.42,17.38,XY24,NTT,MISC
37,HINDUNILVR,2413.81,2723.00,-3.96,H-LC,8.23,301795.0,31448.0,3169.0,2.39,11.63,1.05,12.81,24.0,9.92,2.27,26.22,XY25,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.00
1,25,41.69
2,50,71.34


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.20
LC    36.53
MC    26.30
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.34
X40      14.44
X40N     13.94
XY25     11.99
XR        8.98
AR        8.47
OX40N     7.36
X5K       2.27
MH        1.70
X200      1.43
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.30
H-MC    23.42
H-SC    22.45
M-SC    13.20
M-LC     7.19
M-MC     2.55
L-SC     1.55
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,0.62,29.80
IT,12.10,-16.18,74.21
FINANCE,7.98,-22.90,68.34
BANKS,7.44,-15.46,64.48
MISC,5.94,-16.36,83.06
PAINTS,5.53,-18.98,42.43
ELECTRICAL,4.72,-11.99,60.49
AUTO,4.58,-11.28,59.69
AC,3.68,1.04,32.47


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2870199.0,23
XR,1102194.0,13
AR,1084720.0,10
X40,818116.0,11
X40N,514042.0,11
OX40N,480037.0,10
XY25,454276.0,8
SR,196426.0,2
X5K,137913.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2792725.0,27
M-SC,1492131.0,17
H-MC,1265838.0,18
H-LC,1142520.0,19
M-LC,424946.0,5
M-MC,338827.0,2
L-SC,256316.0,3
L-MC,59227.0,1
L-LC,45634.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      945171.0      6
M-SC       XY24      870233.0      7
H-SC       AR        662497.0      5
           XR        530170.0      6
H-LC       X40       509477.0      6
H-MC       XY24      487704.0      5
M-MC       XY24      338827.0      2
H-LC       AR        234860.0      2
H-MC       X40       223469.0      4
H-SC       OX40N     207462.0      4
H-MC       XY25      197980.0      2
H-SC       SR        196426.0      2
H-LC       X40N      195613.0      5
H-SC       X40N      192948.0      3
M-LC       XY24      178038.0      2
L-SC       XR        166263.0      2
M-SC       AR        157200.0      2
           OX40N     131271.0      4
M-LC       X5K       129631.0      1
H-MC       X40N      125481.0      3
M-SC       XY25      124163.0      1
           XR        124094.0      2
H-MC       XR        118392.0      1
M-LC       XR        104048.0      1
L-SC       OX40N      90053.0      1
M-SC       X40        85170.0      1
H-LC       XY25       73270.0      3
           X200       70792.0      1
L-MC       XR         59227.0      1
H-SC       MH         58051.0      1
H-MC       OX40N      51251.0      1
H-LC       XY24       50226.0      1
L-LC       XY25       45634.0      1
H-MC       MH         31398.0      1
           AR         30163.0      1
M-LC       XY25       13229.0      1
H-LC       X5K         8282.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
